In [32]:
import torch
import torch.nn as nn
import torch.utils.data

import torchsummary

from tqdm import tqdm
import pandas as pd
import copy

import plotly.express as ep

from dataloader import ECGDataset

In [33]:
BATCH_SIZE = 100
LR = 1e-3
EPOCHS = 10


device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [34]:
trainset = ECGDataset()
testset = ECGDataset()

trainloader = torch.utils.data.DataLoader(trainset, BATCH_SIZE,True, pin_memory=True)
testloader = torch.utils.data.DataLoader(testset, BATCH_SIZE,True, pin_memory=True)


In [35]:
class Network(nn.Module):
    def __init__(self,number_of_classes, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.layers = nn.Sequential(
            nn.Conv1d(in_channels=12,out_channels=128,kernel_size=80,stride=64),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(in_channels=128,out_channels=256,kernel_size=11,stride=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Conv1d(in_channels=256,out_channels=128,kernel_size=7,stride=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),
            nn.Linear(in_features=128,out_features=64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(in_features=64,out_features=number_of_classes),
        )
    def forward(self, x):
        return self.layers(x)

In [36]:
model = Network(10)
model = model.to(device)


criterum = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(),LR,weight_decay=1e-5) 
optimizer = torch.optim.SGD(model.parameters(),LR*10,momentum=0.9,weight_decay=1e-5) 

lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer,max_lr=LR,total_steps=EPOCHS)

In [37]:
def train(model, dataloader):
    running_acc = 0
    running_loss = 0
    model.train()
    with tqdm(enumerate(dataloader)) as t:
        t.set_description_str('Train')
        for i,(data,labels) in t:
            # reset
            optimizer.zero_grad(True)

            # Data transfere
            data = data.to(device,non_blocking=True)
            labels = labels.to(device,non_blocking=True)

            # network inference
            target = model(data)

            # loss
            loss = criterum(target,labels)
            running_loss += loss.item()

            # backprobagation
            loss.backward()
            optimizer.step()

            # accuracy calculation
            running_acc += (torch.argmax(target,-1)==labels).sum().item()

            t.set_postfix({'loss':running_loss/(BATCH_SIZE*(i+1)),'acc':running_acc/(BATCH_SIZE*(i+1))})
    return {
        'loss':running_loss/(len(dataloader)*BATCH_SIZE),
        'acc':running_acc/(len(dataloader)*BATCH_SIZE),
        }

def test(model, dataloader):
    running_acc = 0
    running_loss = 0
    model.eval()
    with tqdm(enumerate(dataloader)) as t:
        t.set_description_str('Test ')
        for i,(data,labels) in t:
            # Data transfere
            data = data.to(device,non_blocking=True)
            labels = labels.to(device,non_blocking=True)

            # network inference
            target = model(data)

            # loss
            loss = criterum(target,labels)
            running_loss += loss.item()

            # accuracy calculation
            running_acc += (torch.argmax(target,-1)==labels).sum().item()

            t.set_postfix({'loss':running_loss/(BATCH_SIZE*(i+1)),'acc':running_acc/(BATCH_SIZE*(i+1))})
    return {
        'loss':running_loss/(len(dataloader)*BATCH_SIZE),
        'acc':running_acc/(len(dataloader)*BATCH_SIZE),
        }



In [38]:
stats = []

best_test = 0
best_model = None
for i in range(EPOCHS):
    train_dict = train(model,trainloader)
    test_dict = test(model,testloader)

    train_dict = dict([('Train_'+key,value) for key,value in train_dict.items()])
    test_dict = dict([('Test_'+key,value) for key,value in test_dict.items()])


    stats.append(train_dict|test_dict|{'Epoch':i,'LR':lr_scheduler.get_lr()[0]})
    if test_dict['Test_acc']>best_test:
        best_test = test_dict['Test_acc']
        best_model = copy.deepcopy(model)

    lr_scheduler.step()
stats = pd.DataFrame(stats)
torch.save(best_model.state_dict(),'best_model_state_dict.pt')
torch.save(best_model,'best_model.pt')

best_model_load = Network(10)
best_model_load.load_state_dict(torch.load('best_model_state_dict.pt')) 


Train: 1it [00:00,  4.45it/s, loss=0.0237, acc=0.06]
Test : 1it [00:00, 15.71it/s, loss=0.0231, acc=0.11]
/home/tweak/sources/BSC/.env/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:1761: UserWarning:

To get the last learning rate computed by the scheduler, please use `get_last_lr()`.

Train: 1it [00:00,  5.48it/s, loss=0.0237, acc=0.06]
Test : 1it [00:00, 11.47it/s, loss=0.0231, acc=0.11]
Train: 1it [00:00,  5.35it/s, loss=0.0234, acc=0.1]
Test : 1it [00:00, 12.34it/s, loss=0.0231, acc=0.09]
Train: 1it [00:00,  6.04it/s, loss=0.0224, acc=0.12]
Test : 1it [00:00,  9.40it/s, loss=0.0231, acc=0.06]
Train: 1it [00:00,  4.99it/s, loss=0.0213, acc=0.2]
Test : 1it [00:00, 13.16it/s, loss=0.0231, acc=0.07]
Train: 1it [00:00,  5.77it/s, loss=0.0203, acc=0.28]
Test : 1it [00:00, 11.63it/s, loss=0.0231, acc=0.07]
Train: 1it [00:00,  5.57it/s, loss=0.0195, acc=0.33]
Test : 1it [00:00, 15.30it/s, loss=0.0231, acc=0.07]
Train: 1it [00:00,  6.37it/s, loss=0.019, acc=0.41]
Test : 1it [00:0

In [39]:
torchsummary.summary(model,(12,5000))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 10]                  --
|    └─Conv1d: 2-1                       [-1, 128, 77]             123,008
|    └─BatchNorm1d: 2-2                  [-1, 128, 77]             256
|    └─ReLU: 2-3                         [-1, 128, 77]             --
|    └─Conv1d: 2-4                       [-1, 256, 67]             360,704
|    └─BatchNorm1d: 2-5                  [-1, 256, 67]             512
|    └─ReLU: 2-6                         [-1, 256, 67]             --
|    └─Conv1d: 2-7                       [-1, 128, 61]             229,504
|    └─BatchNorm1d: 2-8                  [-1, 128, 61]             256
|    └─ReLU: 2-9                         [-1, 128, 61]             --
|    └─AdaptiveAvgPool1d: 2-10           [-1, 128, 1]              --
|    └─Flatten: 2-11                     [-1, 128]                 --
|    └─Linear: 2-12                      [-1, 64]                  

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 10]                  --
|    └─Conv1d: 2-1                       [-1, 128, 77]             123,008
|    └─BatchNorm1d: 2-2                  [-1, 128, 77]             256
|    └─ReLU: 2-3                         [-1, 128, 77]             --
|    └─Conv1d: 2-4                       [-1, 256, 67]             360,704
|    └─BatchNorm1d: 2-5                  [-1, 256, 67]             512
|    └─ReLU: 2-6                         [-1, 256, 67]             --
|    └─Conv1d: 2-7                       [-1, 128, 61]             229,504
|    └─BatchNorm1d: 2-8                  [-1, 128, 61]             256
|    └─ReLU: 2-9                         [-1, 128, 61]             --
|    └─AdaptiveAvgPool1d: 2-10           [-1, 128, 1]              --
|    └─Flatten: 2-11                     [-1, 128]                 --
|    └─Linear: 2-12                      [-1, 64]                  

In [40]:
fig = ep.line(stats,x='Epoch',y=['Train_loss','Test_loss'])
fig.show()
fig.write_html('./tmp.html')
fig=ep.line(stats,x='Epoch',y=['Train_acc','Test_acc'])
fig.show()
fig=ep.line(stats,x='Epoch',y=['LR'])
fig.show()